In [1]:
from PIL import Image
from Cryptodome.Cipher import AES
# from pycryto.Cipher import AES
# from Crypto.Util.Padding import pad, unpad
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.Padding import pad, unpad

In [2]:
def string_to_binary(message):
    encodeM = str(message).encode()
    bin_int = int.from_bytes(encodeM, 'big')
    bin_string = bin(bin_int)[2:]
    return "0" + bin_string

In [3]:
def implant_message(pixels, binString):
    # loop through each pixel
    for i in range(len(pixels)):
        for j in range(len(pixels[i])):
            if (binString == ''):
                return pixels
            # get the pixel value this is in binary already
            pixel = pixels[i][j]
            # print(pixel)
            # get first two characters of binString to be implanted into pixel
            binStringC = binString[:2]
            # remove first two characters of binString to the message will be eaten while looping
            binString = binString[2:]
            # get the first 6 character of binC then add the two bits of binStringC to the end
            pixel = pixel[:6] + binStringC
            #set the new pixel value
            pixels[i][j] = pixel
    # return pixels

# def implant_message(pixels, binString):
#     # loop through each pixel
#     length = len(binString)
    
#     for i in range(len(pixels)):
#         # if(binString == ""):
#         #     print
#         #     return pixels

#         for j in range(len(pixels[i])):
#             if (binString == ''):
#                 # print("MESSAGE IMPLANTED")
#                 break
#             # if(len(binString) == length * .75):
#             #     print("25")
#             # if(len(binString) == length * .5):
#             #     print("50")
#             # if(len(binString) == length * 0.25):
#             #     print("75")
#             # get the pixel value this is in binary already
#             pixel = pixels[i][j]
#             # print(pixel)
#             # get first two characters of binString to be implanted into pixel
#             binStringC = binString[:2]
#             # remove first two characters of binString to the message will be eaten while looping
#             binString = binString[2:]
#             # get the first 6 character of binC then add the two bits of binStringC to the end
#             pixel = pixel[:6] + binStringC
#             #set the new pixel value
#             pixels[i][j] = pixel
#     return pixels
 
                

In [4]:
# def retrieve_message(pixels):
#     # create empty list of binary values
#     binList = []
#     # loop through each pixel
#     for i in range(len(pixels)):
#         for j in range(len(pixels[i])):
#             # get the pixel value
#             pixel = pixels[i][j]
#             # print(pixel)
#             # get the last 2 characters of pixel
#             binC = pixel[-2:]
#             # add the the bits to the list
#             binList.append(binC)
#     l = ''.join(binList)
#     return l

In [5]:
# def return_message(msg):
#     test = [msg[i:i+8] for i in range(0, len(msg), 8)]
#     # newArray = np.zeros((len(test), len(test[0]), 3), dtype=np.uint8)
#     apple = ""
#     for t in test:
#         apple += chr(int(t, 2))
#         # chr(int(t, 2)), end=""
#     return apple

In [15]:
def get_message(pixelsWithMessage):
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixelsWithMessage]
    binPixels = [list(pixel[-2:] for pixel in pixel) for pixel in binaryPixels]
    # binPixels
    # iterate through each binPixels and concatenate them into a string
    binString = ''.join([''.join(pixel) for pixel in binPixels])
    #split into 8 bit chunks
    binString = [binString[i:i+8] for i in range(0, len(binString), 8)]
    # print(binString)

    # iterate through each binString and convert to int
    binString = [int(b, 2) for b in binString]
    # binString
    # turn ascii numbers to ascii characters
    binString = [chr(b) for b in binString]

    binString = ''.join(binString)
    return binString

In [7]:
def binary_to_string(binString):
    binary_int = int(binString, 2)  
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    return ascii_text

# def binary_to_string(binString):
#     # get the last 2 bits of each channel of each pixel in binartPixels
#     # binPixels = [list(pixel[-2:] for pixel in pixel) for pixel in binaryPixels]
#     # binPixels
#     # iterate through each binPixels and concatenate them into a string
#     binString = ''.join([''.join(pixel) for pixel in binPixels])
#     #split into 8 bit chunks
#     binString = [binString[i:i+8] for i in range(0, len(binString), 8)]
#     # print(binString)

#     # iterate through each binString and convert to int
#     binString = [int(b, 2) for b in binString]
#     # binString
#     # turn ascii numbers to ascii characters
#     binString = [chr(b) for b in binString]

#     binString = ''.join(binString)
#     return binString

In [8]:
def canEncode(pixels, message):
    LengthM = len(string_to_binary(message))
    LengthP = len(pixels) *3 *2
    print("message length: ", LengthM)
    print("pixels length: ", LengthP)
    if LengthM > LengthP:
        return False, LengthM, LengthP
    return True, LengthM, LengthP

# Hide info and create photo

In [9]:
def hide_Message(Photo1, hiddenMessage, photoName):
    # turn txt file to binary
    binMessage = string_to_binary(hiddenMessage)
    # turn photo to pixel representation
    pixels = Photo1.getdata()
    # turn pixel representation into a list of lists
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixels]
    # implant the message
    newPhotoPixelsBinary = implant_message(binaryPixels, binMessage)
    # turn the binary pixels back into rgb values
    newPhotoPixels = [tuple(int(p,2) for p in pixel) for pixel in newPhotoPixelsBinary]
    # turn the rgb values back into a photo
    newPhoto = Image.new("RGB", Photo1.size)
    newPhoto.putdata(newPhotoPixels)
    # save the photo
    finalFilename = ".".join([photoName, "png"])
    newPhoto.save(finalFilename)



# examples

## no encyrption

In [16]:
Photo1 = Image.open("startImages/Photo1.jpg")
Photo2 = Image.open('startImages/Photo2.jpg')

hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
hiddenMessage = str(hiddenMessage)
# testB = string_to_binary(hiddenMessage)

In [27]:
if canEncode(Photo1.getdata(), hiddenMessage):
    hide_Message(Photo1, hiddenMessage, "encodedImages/notEncypted")

message length:  316144
pixels length:  7588800


In [20]:
notEncryptedMessage = Image.open('encodedImages/notEncypted.png')
pixelsWithMessage = notEncryptedMessage.getdata()
LBSMessage = get_message(pixelsWithMessage)
# LBSMessage
# save the message to a file
with open('decodedMessages/LSBMessage_noEncryption.txt', 'w') as f:
    f.write(LBSMessage)


# with encryption

In [21]:
Photo1 = Image.open("startImages/Photo1.jpg")
hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
hiddenMessage = str(hiddenMessage)

In [22]:
#this works can not decrpyt from it
data = bytes(hiddenMessage, 'utf-8')
key = get_random_bytes(16)
print(" this is the key: ", key)
cipher = AES.new(key, AES.MODE_ECB)
padded = pad(data, AES.block_size)
padded
ciphertext= cipher.encrypt(padded)

 this is the key:  b'L\xe4\x8ey\x9d\xd9>\x85\xec\xfe\xd4W\xf2\xf1\x83~'


test to make sure decrypts correctly

In [23]:
decipher = AES.new(key, AES.MODE_ECB)
decrypted_test = unpad(decipher.decrypt(ciphertext), 16)
# print(decrypted_test)

In [28]:
# hide the message in the photo
if canEncode(Photo1.getdata(), ciphertext):
    hide_Message(Photo1, ciphertext, "encodedImages/encrypted")

message length:  316144
pixels length:  7588800


In [29]:
hiddenImgEncrypted = Image.open('encodedImages/encrypted.png')
pixelsWithMessageEncrypted = hiddenImgEncrypted.getdata()

In [39]:
pixelsWithMessageEncrypted[0]

(109, 66, 60)

In [40]:
encryptedMsg = get_message(pixelsWithMessageEncrypted)
# encryptedMsg

In [41]:
decrypt_test = bytes(encryptedMsg, 'utf-8')
decipher = AES.new(key, AES.MODE_ECB)
decrypted_test = unpad(decipher.decrypt(decrypt_test), 16)
# decrypted_test = decipher.decrypt(pad(encrypted_test,16))
# decrypted_test = unpad(decrypted_test, 16)
# decrypted_test = decrypted_test.decode('utf-8')
print(decrypted_test)

ValueError: Data must be aligned to block boundary in ECB mode

pixelsWithEncryption[0][0]

In [118]:
# def msg_encrypt(msg, cipher):
#     if(len(msg)%16 != 0):
#         msg = msg + " " * (16 - len(msg)%16)
#     msg = msg.encode()
#     crypt_msg = cipher.encrypt(msg)
#     return crypt_msg
# test

In [95]:
# def msg_decrypt(msg, cipher):
#     msg = cipher.decrypt(msg)
#     # msg = msg.decode()
#     return msg

In [72]:
# key = get_random_bytes(16)
# cipher = AES.new(key, AES.MODE_ECB)
# encp_msg = msg_encrypt(hiddenMessage, cipher)

In [76]:
type(encp_msg)

bytes

In [32]:
# test = cipher.decrypt(ciphertext)
# print(test)

In [81]:
# cast ciphertext to string
# ciphertext = str(ciphertext)

canEncode(Photo1.getdata(), ciphertext)

message length:  316048
pixels length:  7588800


(True, 316048, 7588800)

In [82]:
hide_Message(Photo1, encp_msg, "a")

In [83]:
hiddenMessagePhoto = Image.open('a.png')
# x = retrieve_Message(hiddenMessagePhoto)
# 

In [61]:
ciphertext[-1:]

b'\xc3'

In [87]:
pixelsWithMessage = hiddenMessagePhoto.getdata()
# turn pixel representation into a list of lists
# binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixelsWithMessage]

test = get_message(pixelsWithMessage)

# test

ValueError: invalid literal for int() with base 2: b'\xf0\xecs\xea,\xa8\x9fa\x19\xf7\xb9\xb9=\x95`M\n\xe9r\xd4\xcb\xc8\xe0\xf9\xb3\xa2\x89\x03[\xb7T1\xff\xc1\xef\xe6g"\xf6\x15\x0ec\x1cn]\xc5c[\xe6O\x01\xecGr\xb2i\xd6\x05\x97\x7fb\x95\xed\xfd0\'\xd8\xe

In [48]:
# pad(test, 16)

In [63]:
# encrypted_test

In [116]:
# take x and decrypt it
encrypted_test = bytes(test, 'utf-8')
decipher = AES.new(key, AES.MODE_ECB)
decrypted_test = unpad(decipher.decrypt(encrypted_test),16)
# decrypted_test = decipher.decrypt(pad(encrypted_test,16))
# decrypted_test = unpad(decrypted_test, 16)
# decrypted_test = decrypted_test.decode('utf-8')
print(decrypted_test)

ValueError: Input strings must be a multiple of 16 in length

In [ ]:
# def retrieve_message(pixels):
#     # create empty list of binary values
#     binList = []
#     # loop through each pixel
#     for i in range(len(pixels)):
#         for j in range(len(pixels[i])):
#             # get the pixel value
#             pixel = pixels[i][j]
#             # print(pixel)
#             # get the last 2 characters of pixel
#             binC = pixel[-2:]
#             # add the the bits to the list
#             binList.append(binC)
#     l = ''.join(binList)
#     return l

In [100]:

# print(retrieve_Message(hiddenMessagePhoto))
#save the message to a text file
with open('test.txt', 'w') as output:
    output.write(x)

In [115]:
# take x and decrypt it
encrypted_test = bytes(test, 'utf-8')
decipher = AES.new(key, AES.MODE_ECB)
decrypted_test = unpad(decipher.decrypt(ciphertext), 16)
# decrypted_test = decipher.decrypt(pad(encrypted_test,16))
# decrypted_test = unpad(decrypted_test, 16)
# decrypted_test = decrypted_test.decode('utf-8')
print(decrypted_test)

b"We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\nThe Constitutional Convention\nArticle I\nSection 1: Congress\n\nAll legislative Powers herein granted shall be vested in a Congress of the United States, which shall consist of a Senate and House of Representatives.\nSection 2: The House of Representatives\n\nThe House of Representatives shall be composed of Members chosen every second Year by the People of the several States, and the Electors in each State shall have the Qualifications requisite for Electors of the most numerous Branch of the State Legislature.\n\nNo Person shall be a Representative who shall not have attained to the Age of twenty five Years, and been seven Years a Citizen of the United State